# Visale - Action Logement

À destination des jeunes de moins de 30 ans ou des salariés nouvellement embauchés ayant besoin d'un cautionnement pour un logement.

* service-public.fr : https://www.service-public.fr/particuliers/vosdroits/F33453


## Situation à analyser

In [68]:
situation_18yo_moving_away = {
    "id": "18yo-moving-away",
    "description": "Un jeune de 18 ans déménage pour des études à l'université",
    "answers": {
        "statut-professionnel": "etudiant",  # activite
        "situation-professionnelle": "sans-emploi",  # dispatchSituationProfessionnelle
        "etudiant-mobilite": "parcoursup-nouvelle-region",  # dispatchEtudiantMobilite
        "boursier": True,  # boursier
        "date-naissance": "2007-03-01",  # date_naissance
        "handicap": False,  # handicap
        "statut-marital": "celibataire",  # statut_marital
        "code-postal-nouvelle-ville": "75101",  # depcom
        "situation-logement": "locataire",  # dispatchSituationLogement
        "type-logement": "logement-meuble",  # dispatchTypeLogement
        "logement-conventionne": True,  # logement_conventionne
        "colocation": False,  # coloc
        "logement-parente-proprietaire": False,  # proprietaire_proche_famille
        "nombre-personnes-logement": 1,  # 🔥 exclude: True
        "loyer-montant-mensuel": 700,  # loyer
        "loyer-montant-charges": 100,  # charges_locatives
        "loyer-difficile-payer": True,  # exclude: True
        "type-revenus": [
            "aucun-autres-revenus"
        ],
        "confirmation-end": [
            "confirmation-end-oui"
        ]
    },
    "questionsToApi": [
        "locapass-eligibilite",
        "mobilite-master-1",
        "mobilite-parcoursup",
        "aide-personnalisee-logement",
        "garantie-visale-eligibilite",
        "garantie-visale"
    ],
    "results": {
        "locapass": 1200,
        "locapass-eligibilite": True,
        "mobilite-master-1": 0,
        "mobilite-master-1-eligibilite": False,
        "mobilite-parcoursup": 500,
        "mobilite-parcoursup-eligibilite": True,
        "aide-personnalisee-logement": 327,
        "aide-personnalisee-logement-eligibilite": True,
        "garantie-visale": 800,
        "garantie-visale-eligibilite": True
    }
}

to_test = situation_18yo_moving_away

In [69]:
period = '2025-01'

def dispatch_situation_logement(answer_situation_logement: str | None):
	if (answer_situation_logement == 'locataire'):
		# TypesStatutOccupationLogement: 'locataire_foyer', 'locataire_hlm', 'locataire_meuble', 'locataire_vide'
		# TODO ici et aides-simulateur-front : prendre en compte 'type-logement' ("logement-foyer"...)
		return 'locataire_vide'
	
	if (answer_situation_logement == 'proprietaire'):
		# TypesStatutOccupationLogement: 'proprietaire' (could also be a subset: 'primo_accedant')
		return 'proprietaire'
	
	if (answer_situation_logement == 'heberge'):
		return 'loge_gratuitement'

	if (answer_situation_logement == 'sans-domicile'):
		return 'sans_domicile'
	
	return 'non_renseigne'


def dispatch_to_activite(
		answer_statut_professionnel: str | None,
		answer_situation_professionnelle: str | None
		):
	
	if answer_statut_professionnel != None:
		return answer_statut_professionnel

	if (answer_situation_professionnelle == 'salarie-hors-alternance'):
		# TypesActivite possible values: https://legislation.fr.openfisca.org/activite
		return 'actif'  # 'activite'

	if (answer_situation_professionnelle == 'sans-emploi'):
		# TypesActivite possible values: https://legislation.fr.openfisca.org/activite
		return 'chomeur'  # 'activite'



In [70]:
from json import dumps


# TODO autres revenus
# bourse_criteres_sociaux
# salaire_imposable
# chomage_imposable
# rpns_imposables

# décrit la situation au format d'une requête d'API web, sans les variables à calculer
situation_garantie_visale = {
    "individus": {
        "usager": {
            "date_naissance": {
                "ETERNITY": to_test["answers"]["date-naissance"]
            },
            "handicap": {
                period: to_test["answers"]["handicap"] if to_test["answers"]["handicap"] else False
            },
            "statut_marital": {
                period: to_test["answers"]["statut-marital"] if to_test["answers"]["statut-marital"] else "celibataire"
            },
            "activite": {
                period: dispatch_to_activite(to_test["answers"]["statut-professionnel"], to_test["answers"]["situation-professionnelle"])
            },
            "stagiaire": {
                period: to_test["answers"]["situation-professionnelle"] == "stage"
            },
            "alternant": {
                period: to_test["answers"]["situation-professionnelle"] == "alternance"
            },
            "sortie_academie": {
                period: to_test["answers"]["etudiant-mobilite"] == "parcoursup-nouvelle-region"  # false si etudiant-mobilite == 'pas-de-mobilite'
            },
            "sortie_region_academique": {
                period: to_test["answers"]["etudiant-mobilite"] == "master-nouvelle-zone"  # false si etudiant-mobilite == 'pas-de-mobilite'
            }
        }
    },
    "foyers_fiscaux": {
        "foyer_fiscal_1": {
            "declarants": [
                "usager"
            ],
            "personnes_a_charge": []
        }
    },
    "menages": {
        "menage_1": {
            "conjoint": [],
            "enfants": [],
            "personne_de_reference": [
                "usager"
            ],
            "depcom": {
                # TODO conserver le code insee dans les inputs du front
                # TODO en attendant, convertir ici de code postal vers code insee ?
                period: to_test["answers"]["code-postal-nouvelle-ville"] if to_test["answers"]["code-postal-nouvelle-ville"] else ""
            },
            "statut_occupation_logement": {
                # TODO adapter la valeur
                period: dispatch_situation_logement(to_test["answers"]["situation-logement"])
            },
            "loyer": {
                period: to_test["answers"]["loyer-montant-mensuel"] if to_test["answers"]["loyer-montant-mensuel"] else 0.0
            },
            "charges_locatives": {
                period: to_test["answers"]["loyer-montant-charges"]
            },
            "coloc": {
                period: to_test["answers"]["colocation"]
            },
            "logement_conventionne": {
                period: to_test["answers"]["logement-conventionne"]
            }
        }
    },
    "familles": {
        "famille_1": {
            "enfants": [],
            "parents": [
                "usager"
            ],
            "proprietaire_proche_famille": {
                period: to_test["answers"]["logement-parente-proprietaire"]
            }
        }
    }
}

print(dumps(situation_garantie_visale, indent=2))

{
  "individus": {
    "usager": {
      "date_naissance": {
        "ETERNITY": "2007-03-01"
      },
      "handicap": {
        "2025-01": false
      },
      "statut_marital": {
        "2025-01": "celibataire"
      },
      "activite": {
        "2025-01": "etudiant"
      },
      "stagiaire": {
        "2025-01": false
      },
      "alternant": {
        "2025-01": false
      },
      "sortie_academie": {
        "2025-01": true
      },
      "sortie_region_academique": {
        "2025-01": false
      }
    }
  },
  "foyers_fiscaux": {
    "foyer_fiscal_1": {
      "declarants": [
        "usager"
      ],
      "personnes_a_charge": []
    }
  },
  "menages": {
    "menage_1": {
      "conjoint": [],
      "enfants": [],
      "personne_de_reference": [
        "usager"
      ],
      "depcom": {
        "2025-01": "75101"
      },
      "statut_occupation_logement": {
        "2025-01": "locataire_vide"
      },
      "loyer": {
        "2025-01": 700
      },
      "ch

In [71]:
from openfisca_core.simulation_builder import SimulationBuilder
from openfisca_france import FranceTaxBenefitSystem


tbs = FranceTaxBenefitSystem()

sb = SimulationBuilder()
simulation = sb.build_from_entities(tbs, situation_garantie_visale)

visale_variables = ['visale_eligibilite', 'visale_base_ressources', 'visale_montant_max']
# visale_variables_entities = ['menages', 'menages', 'menages']
# visale_variables_groupes = ['menage_1', 'menage_1', 'menage_1']

for variable_name in visale_variables:
    print(f"{variable_name}: {simulation.calculate(variable_name, period)}")

visale_eligibilite: [False]
visale_base_ressources: [121.7055]
visale_montant_max: [800.]


In [72]:
# TODO fix assert to_test["results"]["garantie-visale-eligibilite"] == simulation.calculate("visale_eligibilite", period)[0]
assert to_test["results"]["garantie-visale"] == simulation.calculate("visale_montant_max", period)[0]

## Éligibilité

### Éligibilité locataire

> Source : https://www.visale.fr/visale-pour-les-locataires/eligibilite/

* J’ai entre 18 et 30 ans
* Je suis salarié(e) de plus de 30 ans : 
    * Embauché(e) depuis moins de 6 mois (hors CDI confirmé)
    * OU Gagnant jusqu’à 1500€ nets/mois
    * OU En mobilité professionnelle
    * OU En possession d’une promesse d’embauche de moins de 3 mois
* Je suis éligible au bail mobilité.
* Je suis logé(e) par un organisme d’intermédiation locative


> Si jeune de moins de 30 ans jusqu'au 31ème anniversaire d'après [ce PDF https://www.morbihan.gouv.fr](https://www.morbihan.gouv.fr/contenu/telechargement/40631/297616/file/VISALE_DEPLIANT_LOCATAIRE_120916.pdf)

In [74]:
age = simulation.calculate('age', period)
age[0]


17

### Éligibilité logement

> Source : https://www.visale.fr/visale-pour-les-locataires/eligibilite/

* Peut être meublé ou non
* Doit être ma résidence principale
* Doit avoir un loyer (charges comprises) de 1500€ maximum en Île de France ou de 1300€ maximum sur le reste du territoire
* Doit être décent et respecter le Règlement Sanitaire Départemental (RSD)
* Doit faire l’objet d’un bail (contrat de location)

## Limites

> Source : https://www.visale.fr/wp-content/uploads/2024/08/depliant_visale_locataire_juin24.pdf

Dans la limite de 36 mensualités d’impayés de loyers
et charges pour un logement du parc privé. 
Dans le parc locatif social ou assimilé, prise en charge dans la limite de 9
mois d’impayés de loyers et charges locatives, déduction faite
des aides au logement. 
Dans le parc locatif privé, en cas de
dégradations imputables au locataire, les frais de remise en
état du logement sont couverts, jusqu'à deux mois de loyer et
charges inscrits au bail, en complément du dépôt de garantie
(sauf pour le bail mobilité).

Pour les travailleurs saisonniers, la durée de prise en charge
des impayés est limitée de 1 à 9 mois quel que soit le parc
locatif. Pour les dégradations locatives : prise en charge jusqu’à
deux mois de loyer et charges inscrits au bail, complétée le
cas échéant d’une prise en charge des dommages mobiliers, à
hauteur d’un mois de loyer.